## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import re

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)

In [2]:
def tokenize_text(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

In [3]:
raw_train = []


with open('news_train.txt', 'r') as f:
  for line in f:
    label, title, text = line.strip().split("\t")
    raw_train.append((label, title, text))

data_train = []
for label, title, text in raw_train:
  item = {}
  item['label'] = label
  item['title'] = tokenize_text(title)
  item['text'] = [tokenize_text(sentance) for sentance in re.split(r"[.!?]", text) if len(sentance) > 10]
  data_train.append(item)

In [4]:
len(data_train), data_train[0]

(15000,
 {'label': 'sport',
  'text': [['нападающий',
    'вашингтон',
    'кэпиталз',
    'александр',
    'овечкин',
    'передал',
    'детской',
    'хоккейной',
    'школе',
    'автомобиль',
    'полученный',
    'им',
    'после',
    'окончания',
    'матча',
    'всех',
    'звезд',
    'национальной',
    'хоккейной',
    'лиги',
    'нхл'],
   ['об', 'этом', 'сообщается', 'на', 'официальном', 'сайте', 'лиги'],
   ['автомобиль',
    'honda',
    'accord',
    'был',
    'подарен',
    'хоккеисту',
    'по',
    'решению',
    'спонсоров',
    'мероприятия'],
   ['игрок',
    'нхл',
    'пожертвовал',
    'машину',
    'спортивной',
    'школе',
    'nova',
    'cool',
    'cats',
    'special',
    'hockey',
    'inc'],
   ['которая', 'расположена', 'в', 'штате', 'вирджиния'],
   ['овечкин',
    'общается',
    'с',
    '10',
    'летней',
    'девочкой',
    'анной',
    'шоб',
    'с',
    'синдромом',
    'дауна',
    'которая',
    'занимается',
    'в',
    'этой',
    '

In [5]:
raw_test = []


with open('news_test.txt', 'r') as f:
  for line in f:
    label, title, text = line.strip().split("\t")
    raw_test.append((label, title, text))

data_test = []
for label, title, text in raw_test:
  item = {}
  item['label'] = label
  item['title'] = tokenize_text(title)
  item['text'] = [tokenize_text(sentance) for sentance in re.split(r"[.!?]", text) if len(sentance) > 10]
  data_test.append(item)

In [6]:
len(data_test), data_test[0]

(3000,
 {'label': 'culture',
  'text': [['жительница',
    'ямало',
    'ненецкого',
    'автономного',
    'округа',
    'елена',
    'лаптандер',
    'победила',
    'в',
    'первом',
    'всероссийском',
    'песенном',
    'конкурсе',
    'новая',
    'звезда',
    'сообщили',
    'ленте'],
   ['ру', 'организаторы', 'события'],
   ['в',
    'качестве',
    'награды',
    'ей',
    'досталась',
    'статуэтка',
    'в',
    'форме',
    'звезды',
    'и',
    'денежный',
    'приз',
    'один',
    'миллион',
    'рублей'],
   ['по',
    'словам',
    'финалистки',
    'вознаграждение',
    'она',
    'планирует',
    'передать',
    'в',
    'благотворительный',
    'фонд',
    'подари',
    'жизнь'],
   ['три',
    'дополнительных',
    'приза',
    'достались',
    'руслану',
    'ивакину',
    'из',
    'хакасии',
    'фолк',
    'группе',
    'ярилов',
    'зной',
    'из',
    'воронежской',
    'области',
    'и',
    'александру',
    'куулару',
    'из',
    'тывы'],
   ['

In [7]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

for item in tqdm(data_test):
  item['title'] = [stemmer.stem(word) for word in item['title']] 
  item['text'] = [[stemmer.stem(word) for word in sentance] for sentance in item['text']]

for item in tqdm(data_train):
  item['title'] = [stemmer.stem(word) for word in item['title']] 
  item['text'] = [[stemmer.stem(word) for word in sentance] for sentance in item['text']]

In [8]:
data_test[0]

{'label': 'culture',
 'text': [['жительниц',
   'яма',
   'ненецк',
   'автономн',
   'округ',
   'ел',
   'лаптандер',
   'побед',
   'в',
   'перв',
   'всероссийск',
   'песен',
   'конкурс',
   'нов',
   'звезд',
   'сообщ',
   'лент'],
  ['ру', 'организатор', 'событ'],
  ['в',
   'качеств',
   'наград',
   'е',
   'доста',
   'статуэтк',
   'в',
   'форм',
   'звезд',
   'и',
   'денежн',
   'приз',
   'один',
   'миллион',
   'рубл'],
  ['по',
   'слов',
   'финалистк',
   'вознагражден',
   'он',
   'планир',
   'переда',
   'в',
   'благотворительн',
   'фонд',
   'подар',
   'жизн'],
  ['три',
   'дополнительн',
   'приз',
   'доста',
   'руслан',
   'ивакин',
   'из',
   'хакас',
   'фолк',
   'групп',
   'ярил',
   'зно',
   'из',
   'воронежск',
   'област',
   'и',
   'александр',
   'куулар',
   'из',
   'тыв'],
  ['призер',
   'с',
   'помощ',
   'смс',
   'голосован',
   'выбира',
   'зрител',
   'телекана',
   'звезд',
   'котор',
   'транслирова',
   'конкурс'],
  ['в

2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

In [9]:
from gensim.models import Word2Vec

sentences = [item['title'] for item in data_train]
sentences.extend([sentance for item in data_train for sentance in item['text']])

w2v = Word2Vec(sentences, workers=8)

In [10]:
w2v.wv.most_similar(positive=['конкурс'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('соревнован', 0.7557839155197144),
 ('мероприят', 0.7307270765304565),
 ('фестивал', 0.727620542049408),
 ('тэф', 0.7191061973571777),
 ('голосован', 0.7015584707260132),
 ('церемон', 0.6978302001953125),
 ('евровиден', 0.6857688426971436),
 ('финалист', 0.6713295578956604),
 ('выставк', 0.6588515639305115),
 ('состязан', 0.641871452331543)]

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия

In [11]:
max_item_len = 140
labels = ('science', 
          'style', 
          'culture', 
          'life', 
          'economics', 
          'business', 
          'travel', 
          'forces', 
          'media', 
          'sport')

label_indices = {}
for l in labels:
  label_indices[l] = labels.index(l)

In [12]:
X_train = []
y_train = []

for item in tqdm(data_train):
    label = item["label"]
    title = item["title"]
    text = item["text"]

    label_index = label_indices[label]

    word_index = 0
    sent_index = 0
    pos_in_sent = 0

    x = []

    while word_index < max_item_len:
        if word_index < len(title):
            x.append(title[word_index])
            word_index += 1
        else:
            if pos_in_sent < len(text[sent_index]):
                x.append(text[sent_index][pos_in_sent])
                word_index += 1
                pos_in_sent += 1
            elif sent_index < len(text) - 1:
                sent_index += 1
                pos_in_sent = 0
            else:
                x.append("PLACEHOLDER")
                word_index += 1

    X_train.append(" ".join(x))
    y_train.append(label_index)

In [13]:
X_test = []
y_test = []

for item in tqdm(data_test):
    label = item["label"]
    title = item["title"]
    text = item["text"]

    label_index = label_indices[label]

    word_index = 0
    sent_index = 0
    pos_in_sent = 0

    x = []

    while word_index < max_item_len:
        if word_index < len(title):
            x.append(title[word_index])
            word_index += 1
        else:
            if pos_in_sent < len(text[sent_index]):
                x.append(text[sent_index][pos_in_sent])
                word_index += 1
                pos_in_sent += 1
            elif sent_index < len(text) - 1:
                sent_index += 1
                pos_in_sent = 0
            else:
                x.append("PLACEHOLDER")
                word_index += 1

    X_test.append(" ".join(x))
    y_test.append(label_index)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_idf = tfidf.fit_transform(X_train)
X_test_idf = tfidf.transform(X_test)

In [15]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


accuracy = 0.853


In [16]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.796


In [17]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


accuracy = 0.71
